In [1]:
from functions import *
import resources

running


# 流水线说明
**采用五级流水线，暂未考虑指令相关的解决**
## 项目结构与函数实现
- 五级流水线的实现函数分成三个模块：functions，resources,simulator
- functions中定义了CPU的五个阶段，并包含了一些辅助的函数
- resources包含了程序可见的状态，包括寄存器，条件码，内存，PC，程序状态等一些信息。
- simulator是主程序，模拟指令集的运行。其读取COE格式的文件。每条指令执行后可以输出此时CPU的内存、寄存器、状态码、CPU状态等信息。
## 设计思路
指令集采用经典的Y86指令集。五级流水也是经典的取指，译码，执行，访存，写回五个阶段。模拟执行流水线时，利用threading模块的函数来实现流水线每个周期每个阶段的并行。
## test
将以下一段不相关指令加载到内存中准备执行
```
memory_initialization_radix = 16; 
memory_initialization_vector = 
30F00100000000000000,   irmov 0x1 %rax
10, nop
10, nop
10, nop
30F10200000000000000,   irmov 0x2 %rcx
10, nop
10, nop
10, nop
6011,   addq %rcx %rcx
6022,   addq %rdx %rdx
6033,   addq %rbx %rbx
10, nop
10, nop
6001,   addq %rax %rcx
10, nop
10, nop
10, nop
00; halt
```
寄存器值为：
[1, 5, 0, 0, 500, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

观察上面的输出，可见rax为1，rcx为5。指令执行正确

从周期0开始到周期18结束，共取值19条指令。流水线工作正常。

In [2]:
# 加载器，将机器码程序加载到内存中
def loadProgram(file):
    fil = open(file,'r')
    lineno = 0
    address = 0
    while True:
        line = fil.readline()
        lineno += 1
        if (lineno == 1 or lineno == 2):
            continue
        if(line == '' or line == '\n'):
            break
        try:
            instruc = line
            instruc = instruc.strip('\n')
            instruc = instruc.split(',')[0]
            instruc = instruc.split(';')[0]
            instruc = instruc.strip(',')    #指令为字符串类型,十六进制
            instruc = instruc.strip(';')

            print(instruc, len(instruc))
            
            for i in range(0, len(instruc)//2):
                resources.Imem[address] = instruc[2*i : 2*i+2]
                address += 1
        except:
            print(f'File {file} line {lineno} has error')
            pass
    fil.close

In [3]:
loadProgram("test_stall.coe")
print(resources.Imem)
print(resources.reg)
print(resources.PC)

30F00100000000000000 20
30F10100000000000000 20
6001 4
6001 4
6001 4
10 2
10 2
10 2
10 2
00 2
['30', 'F0', '01', '00', '00', '00', '00', '00', '00', '00', '30', 'F1', '01', '00', '00', '00', '00', '00', '00', '00', '60', '01', '60', '01', '60', '01', '10', '10', '10', '10', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00',

In [4]:
def check():
   #print(resources.stat)
   if(resources.stat == 1):
      return 1
   elif(resources.stat == 2):
      print("Error: Halt")
      return 0
   elif(resources.stat == 3):
      print("Error: ilegal address")
      return 0
   elif(resources.stat == 4):
      print("Error: ilegal instruction")
      return 0

instruction_no = 0


结构依赖设计思路：

    采用了两个alu来分别执行两周期的EX（增加资源）
    参数如下
    EX_input_signals1 = (0, 0, 0, 0, 0, 0, 0, 0, None, None)
    EX_input_signals2 = (0, 0, 0, 0, 0, 0, 0, 0, None, None)
    EX_result_signals1 = (0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0)
    EX_result_signals2 = (0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0)
    execute1 = threading.Thread(target=task_execute1)
    execute2 = threading.Thread(target=task_execute2)
    运行时偶数周期执行execute1，奇数周期执行execute2
    偶数周期execute2结束，奇数周期execute1结束

In [5]:
import threading

# 下面这些是来自于上一个周期的信号，也就是这一周期各个阶段接受的输入信号
no = 0
# 用于本周期的计算
IF_stall = 0
ID_stall = 0
EX_bubble = 0
# IF_ID寄存器的状态，即此时刻寄存器的值
IF_ID_status = (1, 0, 0, 0, 0, 0, 0)
# (pre_valP, pre_Cnd, pre_valC, pre_icode, pre_valM, pre_PC)
IF_input_signals = (0, 0, 0, 1, 0, 0)

# (IF_icode, IF_ifun, IF_rA, IF_rB, IF_valP, IF_valC, IF_PC) 
ID_input_signals =(0, 0, 0, 0, 0, 0, 0)

# (ID_icode, ID_ifun, ID_valA, ID_valB, ID_valP, ID_valC, ID_rB, ID_rA, ID_dstE, ID_dstM) 
EX_input_signals1 = (0, 0, 0, 0, 0, 0, 0, 0, None, None)

# (ID_icode, ID_ifun, ID_valA, ID_valB, ID_valP, ID_valC, ID_rB, ID_rA, ID_dstE, ID_dstM) 
EX_input_signals2 = (0, 0, 0, 0, 0, 0, 0, 0, None, None)

# (EX_valE, EX_valP, EX_Cnd,  EX_valC, EX_icode, EX_ifun, EX_valA, EX_rB, EX_rA, EX_dstE, EX_dstM) 
MEM_input_signals = (0, 0, 0, 0, 0, 0, 0, 0, 0, None, None)

# (MEM_valP, MEM_Cnd, MEM_valC, MEM_icode, MEM_valM, MEM_rB, MEM_valE, MEM_ifun, MEM_rA, MEM_dstE, MEM_dstM)  
WB_input_signals = (0, 0, 0, 0, 0, 0, 0, 0, 0, None, None)

# ------------------------------------------------------------------------------------------------
# 下面是本周期计算完成产生的结果信号
# 这样分离是为了避免多线程带来的共享变量临界区

# (IF_icode, IF_ifun, IF_rA, IF_rB, IF_valP, IF_valC, IF_PC)
IF_result_signals = (1, 0, 0, 0, 0, 0, 0)

# (ID_icode, ID_ifun, ID_valA, ID_valB, ID_valP, ID_valC, ID_rB, ID_rA, ID_dstE, ID_dstM)
ID_result_signals =(1, 0, 0, 0, 0, 0, 0, 0, 0, 0)

# (EX_valE, EX_valP, EX_Cnd,  EX_valC, EX_icode, EX_ifun, EX_valA, EX_rB, EX_rA, EX_dstE, EX_dstM)
EX_result_signals1 = (0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0)

# (EX_valE, EX_valP, EX_Cnd,  EX_valC, EX_icode, EX_ifun, EX_valA, EX_rB, EX_rA, EX_dstE, EX_dstM)
EX_result_signals2 = (0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0)

# (MEM_valP, MEM_Cnd, MEM_valC, MEM_icode, MEM_valM, MEM_rB, MEM_valE, MEM_ifun, MEM_rA, MEM_dstE, MEM_dstM)
MEM_result_signals = (0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0)

# (WB_icode, WB_ifun, WB_dstE, WB_dstM)
WB_result_signals = (0, 0, 0, 0)


# -------------------------------------------------------------------------------------------------
# 五个阶段的线程任务函数
# 使用上一个周期的信号进行计算，产生本周期的输出信号
# 在后面的cycle函数中，一个周期的末尾，各个阶段都已经计算完成后，将上个周期的信号值更新为本周期计算产生的新的信号值
def task_fetch():
   global IF_input_signals, IF_result_signals, ID_stall
   IF_result_signals = Fetch(IF_input_signals, ID_stall)
   # TODO
   # 目前未考虑任何分支或者跳转指令
   # debug
   thread_id = threading.current_thread().name
   print(f"thread {thread_id} fetching")

def task_decode():
   global ID_input_signals, ID_result_signals
   ID_result_signals = Decode(ID_input_signals)
   # debug
   thread_id = threading.current_thread().name
   print(f"thread {thread_id} decoding")

def task_execute1():
   global EX_input_signals1, EX_result_signals1
   EX_result_signals1 = Execute(EX_input_signals1)
   # debug
   thread_id = threading.current_thread().name
   print(f"thread {thread_id} executing")

def task_execute2():
   global EX_input_signals2, EX_result_signals2
   EX_result_signals2 = Execute(EX_input_signals2)
   # debug
   thread_id = threading.current_thread().name
   print(f"thread {thread_id} executing")

def task_memory():
   global MEM_input_signals, MEM_result_signals
   MEM_result_signals = Memory(MEM_input_signals)
   # debug
   thread_id = threading.current_thread().name
   print(f"thread {thread_id} memorying")

def task_writeback():
   global WB_input_signals, WB_result_signals
   WB_result_signals = WriteBack(WB_input_signals)
   # debug
   thread_id = threading.current_thread().name
   print(f"thread {thread_id} writingback")


execute1 = threading.Thread(target=task_execute1)
execute2 = threading.Thread(target=task_execute2)
# 模拟一个周期的执行，同一时刻五个阶段（线程）各自执行自己的指令
def cycle():
   # 本周期即将计算的信号和来自上一周期的计算完成的信号
   global IF_result_signals, ID_result_signals, EX_result_signals1, EX_result_signals2, MEM_result_signals
   global IF_input_signals, ID_input_signals, EX_input_signals1, EX_input_signals2, MEM_input_signals, WB_input_signals
   global IF_stall, ID_stall, EX_bubble, IF_ID_status, no
   global execute1, execute2

   # 每个周期开始（模拟时钟上升沿）刷新控制信号和寄存器的值
   # 首先计算冲突检测模块的结果，这里是组合逻辑
   ID_rA = ID_result_signals[7]; ID_rB = ID_result_signals[6];
   if(no%2 == 0): 
      EX_dstM = EX_result_signals1[10]; EX_dstE = EX_result_signals1[9]
   else: 
      EX_dstM = EX_result_signals2[10]; EX_dstE = EX_result_signals2[9]
   MEM_dstM = MEM_result_signals[10]; MEM_dstE = MEM_result_signals[9]
   WB_dstM = WB_result_signals[3]; WB_dstE = WB_result_signals[2]
   HD_signals = (ID_rA, ID_rB, EX_dstM, EX_dstE, MEM_dstM, MEM_dstE, WB_dstM, WB_dstE)
   (ID_stall, EX_bubble, IF_stall) = Hazard_Detection(HD_signals)

   # 更新各个阶段的输入信号的值
   (IF_icode, IF_ifun, IF_rA, IF_rB, IF_valP, IF_valC, IF_pc) = IF_result_signals
   IF_input_signals = (IF_valP, 0, IF_valC, IF_icode, 0, IF_pc)

   # ID的输入来自IF_ID寄存器的输出
   (IF_ID_status, IF_ID_output) = IF_ID(IF_ID_status ,IF_result_signals, ID_stall)
   ID_input_signals = IF_ID_output

   # EX的输入来自ID_EX寄存器
   if(no%2 == 0):
      EX_input_signals1 = ID_EX(ID_result_signals, EX_bubble)
   else:
      EX_input_signals2 = ID_EX(ID_result_signals, EX_bubble)

   if(no%2 == 1):
      MEM_input_signals = EX_result_signals1
   else:
      MEM_input_signals = EX_result_signals2
   WB_input_signals = MEM_result_signals

   # 创建5个线程，每个线程完成一个阶段的工作
   fetch = threading.Thread(target = task_fetch)
   decode = threading.Thread(target = task_decode) 
   if(no%2 == 0):
      execute1 = threading.Thread(target=task_execute1)
   elif(no%2 == 1):
      execute2 = threading.Thread(target=task_execute2)
   memory = threading.Thread(target = task_memory)
   writeback = threading.Thread(target = task_writeback)

   fetch.start()
   decode.start()
   if(no%2 == 0):
      execute1.start()
   elif(no%2 == 1):
      execute2.start()
   memory.start()
   writeback.start()

   fetch.join()
   decode.join()
   # 第0个周期没有ex线程被join
   if(no == 0):
      pass
   elif(no%2 == 1):
      execute1.join()
   elif(no%2 == 0):
      execute2.join()
   memory.join()
   writeback.join()

   if check() == False:
      return False

   # debug
   print(resources.reg)

   return True

In [6]:
while True:
    if not cycle():
        break
    print(f"current cycle:{no}")
    no += 1
    print()

# print(resources.mem)
print(resources.reg)

thread Thread-7 fetching
thread Thread-8 decoding
thread Thread-9 executing
thread Thread-10 memorying
thread Thread-11 writingback
[0, 0, 0, 0, 500, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
current cycle:0

thread Thread-12 fetching
thread Thread-13 decoding
thread Thread-14 executing
thread Thread-15 memorying
thread Thread-16 writingback
[0, 0, 0, 0, 500, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
current cycle:1

thread Thread-17 fetching
thread Thread-18 decoding
thread Thread-19 executing
thread Thread-20 memorying
thread Thread-21 writingback
[0, 0, 0, 0, 500, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
current cycle:2

thread Thread-22 fetching
thread Thread-23 decoding
thread Thread-24 executing
thread Thread-25 memorying
thread Thread-26 writingback
[0, 0, 0, 0, 500, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
current cycle:3

thread Thread-27 fetching
thread Thread-28 decoding
thread Thread-29 executing
thread Thread-30 memorying
thread Thread-31 writingback
[0, 0, 0, 0, 500, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
curre